In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import statsmodels.formula.api as smf
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, matthews_corrcoef


from matplotlib.lines import Line2D
from matplotlib.patches import Patch

In [76]:
phase2_df1=pd.read_csv('phase2_df(null).csv',low_memory=False)

In [71]:
phase2_df1[(phase2_df1 ['light_regime']=='10min-10min')&(phase2_df1 ['plate']=='32v1')]

,plate,measurement,start_date,light_regime,dark_threshold,light_threshold,num_frames,i,j,fv_fm,...,measurement_time_173,measurement_time_174,measurement_time_175,measurement_time_176,measurement_time_177,well_id,mutant_ID,feature,mutated_genes,num_mutations
11873,32v1,M6,2024-08-09,10min-10min,15.521382,21.828480,180,0,1,0.607573,...,NaN,NaN,NaN,NaN,NaN,A02,LMJ.RY0402.039093,CDS,Cre13.g588959,1.0
11874,32v1,M6,2024-08-09,10min-10min,15.521382,21.828480,180,0,2,0.579800,...,NaN,NaN,NaN,NaN,NaN,A03,LMJ.RY0402.039746,CDS,Cre02.g080900,1.0
11875,32v1,M6,2024-08-09,10min-10min,15.521382,21.828480,180,0,3,0.632256,...,NaN,NaN,NaN,NaN,NaN,A04,LMJ.RY0402.039953,MULTIPLE_SPLICE_VARIANTS,Cre02.g095137,1.0
11876,32v1,M6,2024-08-09,10min-10min,15.521382,21.828480,180,0,4,0.536324,...,NaN,NaN,NaN,NaN,NaN,A05,LMJ.RY0402.040368,intron,Cre06.g278251,1.0
11877,32v1,M6,2024-08-09,10min-10min,15.521382,21.828480,180,0,5,0.495094,...,NaN,NaN,NaN,NaN,NaN,A06,LMJ.RY0402.040792,CDS,Cre02.g098750,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12634,32v1,M6,2024-11-20,10min-10min,15.444170,21.656998,172,15,19,0.652275,...,NaN,NaN,NaN,NaN,NaN,P20,LMJ.RY0402.048478,CDS,Cre06.g270450,1.0
12635,32v1,M6,2024-11-20,10min-10min,15.444170,21.656998,172,15,20,0.653241,...,NaN,NaN,NaN,NaN,NaN,P21,LMJ.RY0402.065081,intron,Cre12.g493050,1.0
12636,32v1,M6,2024-11-20,10min-10min,15.444170,21.656998,172,15,21,0.617321,...,NaN,NaN,NaN,NaN,NaN,P22,LMJ.RY0402.165706,CDS,Cre02.g098350,1.0
12637,32v1,M6,2024-11-20,10min-10min,15.444170,21.656998,172,15,22,0.612281,...,NaN,NaN,NaN,NaN,NaN,P23,LMJ.RY0402.068059,CDS,Cre02.g098350,1.0


In [63]:
plates = ['32v1', '32v2', '32v3']
phase2_df1[phase2_df1['plate'].isin(plates)].shape

(9575, 726)

In [55]:
subset=phase2_df1[
    ((phase2_df1['light_regime'] == '10min-10min') & (phase2_df1['y2_87'].notnull()))
]
subset

,plate,measurement,start_date,light_regime,dark_threshold,light_threshold,num_frames,i,j,fv_fm,...,measurement_time_173,measurement_time_174,measurement_time_175,measurement_time_176,measurement_time_177,well_id,mutant_ID,feature,mutated_genes,num_mutations
1533,30v2,M6,2024-06-22,10min-10min,16.012451,22.305840,180,0,2,0.707743,...,NaN,NaN,NaN,NaN,NaN,A03,LMJ.RY0402.189784,intron,Cre01.g001800,1.0
1534,30v2,M6,2024-06-22,10min-10min,16.012451,22.305840,180,0,3,0.610486,...,NaN,NaN,NaN,NaN,NaN,A04,LMJ.RY0402.229006,MULTIPLE_SPLICE_VARIANTS,"Cre13.g583250,Cre07.g341900",2.0
1535,30v2,M6,2024-06-22,10min-10min,16.012451,22.305840,180,0,4,0.693976,...,NaN,NaN,NaN,NaN,NaN,A05,LMJ.RY0402.164384,intron,Cre09.g392500,1.0
1536,30v2,M6,2024-06-22,10min-10min,16.012451,22.305840,180,0,5,0.609684,...,NaN,NaN,NaN,NaN,NaN,A06,LMJ.RY0402.163529,CDS,Cre01.g025400,1.0
1537,30v2,M6,2024-06-22,10min-10min,16.012451,22.305840,180,0,6,0.688343,...,NaN,NaN,NaN,NaN,NaN,A07,LMJ.RY0402.180444,MULTIPLE_SPLICE_VARIANTS,Cre04.g219700,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17230,31v1,M7,2024-08-02,10min-10min,15.623226,22.046017,180,15,19,0.758742,...,NaN,NaN,NaN,NaN,NaN,P20,NaN,NaN,NaN,NaN
17231,31v1,M7,2024-08-02,10min-10min,15.623226,22.046017,180,15,20,0.740380,...,NaN,NaN,NaN,NaN,NaN,P21,NaN,NaN,NaN,NaN
17232,31v1,M7,2024-08-02,10min-10min,15.623226,22.046017,180,15,21,0.781605,...,NaN,NaN,NaN,NaN,NaN,P22,NaN,NaN,NaN,NaN
17233,31v1,M7,2024-08-02,10min-10min,15.623226,22.046017,180,15,22,0.726807,...,NaN,NaN,NaN,NaN,NaN,P23,NaN,NaN,NaN,NaN


In [57]:
subset[subset['plate']=='32v1']

,plate,measurement,start_date,light_regime,dark_threshold,light_threshold,num_frames,i,j,fv_fm,...,measurement_time_173,measurement_time_174,measurement_time_175,measurement_time_176,measurement_time_177,well_id,mutant_ID,feature,mutated_genes,num_mutations
11873,32v1,M6,2024-08-09,10min-10min,15.521382,21.82848,180,0,1,0.607573,...,NaN,NaN,NaN,NaN,NaN,A02,LMJ.RY0402.039093,CDS,Cre13.g588959,1.0
11874,32v1,M6,2024-08-09,10min-10min,15.521382,21.82848,180,0,2,0.579800,...,NaN,NaN,NaN,NaN,NaN,A03,LMJ.RY0402.039746,CDS,Cre02.g080900,1.0
11875,32v1,M6,2024-08-09,10min-10min,15.521382,21.82848,180,0,3,0.632256,...,NaN,NaN,NaN,NaN,NaN,A04,LMJ.RY0402.039953,MULTIPLE_SPLICE_VARIANTS,Cre02.g095137,1.0
11876,32v1,M6,2024-08-09,10min-10min,15.521382,21.82848,180,0,4,0.536324,...,NaN,NaN,NaN,NaN,NaN,A05,LMJ.RY0402.040368,intron,Cre06.g278251,1.0
11877,32v1,M6,2024-08-09,10min-10min,15.521382,21.82848,180,0,5,0.495094,...,NaN,NaN,NaN,NaN,NaN,A06,LMJ.RY0402.040792,CDS,Cre02.g098750,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12251,32v1,M6,2024-08-09,10min-10min,15.521382,21.82848,180,15,19,0.634011,...,NaN,NaN,NaN,NaN,NaN,P20,LMJ.RY0402.048478,CDS,Cre06.g270450,1.0
12252,32v1,M6,2024-08-09,10min-10min,15.521382,21.82848,180,15,20,0.703272,...,NaN,NaN,NaN,NaN,NaN,P21,LMJ.RY0402.065081,intron,Cre12.g493050,1.0
12253,32v1,M6,2024-08-09,10min-10min,15.521382,21.82848,180,15,21,0.598009,...,NaN,NaN,NaN,NaN,NaN,P22,LMJ.RY0402.165706,CDS,Cre02.g098350,1.0
12254,32v1,M6,2024-08-09,10min-10min,15.521382,21.82848,180,15,22,0.554836,...,NaN,NaN,NaN,NaN,NaN,P23,LMJ.RY0402.068059,CDS,Cre02.g098350,1.0


In [61]:
plates = ['32v1', '32v2', '32v3']
phase2_df1[phase2_df1['plate'].isin(plates)].shape

(9202, 726)

In [72]:
phase2_df1 = phase2_df1[
    ~((phase2_df1['light_regime'] == '10min-10min') & (phase2_df1['y2_87'].notnull()))
]

In [59]:
subset=phase2_df1[
    ((phase2_df1['light_regime'] == '10min-10min') & (phase2_df1['y2_87'].notnull()))
]
subset

,plate,measurement,start_date,light_regime,dark_threshold,light_threshold,num_frames,i,j,fv_fm,...,measurement_time_173,measurement_time_174,measurement_time_175,measurement_time_176,measurement_time_177,well_id,mutant_ID,feature,mutated_genes,num_mutations


In [67]:
phase2_df1 .shape

(36064, 726)

In [64]:
phase2_df1 .shape

(37534, 726)

In [75]:
# Step 1: Filter data for the three plates
pd.set_option('display.max_rows', 80)
plates = ['32v1', '32v2', '32v3']
df_32v =phase2_df1[phase2_df1['plate'].isin(plates)]

# Step 2: Count rows per (plate, mutant_ID, mutated_genes, light_regime)
group_counts = (
    df_32v.groupby(['plate', 'light_regime', 'mutant_ID', 'mutated_genes'])
    .size()
    .reset_index(name='count')
)

summary = (
    group_counts.groupby(['light_regime','plate', 'count'])
    .size()
    .reset_index(name='n_mutants')
)

# Optional: Sort for easier reading
summary = summary.sort_values(by=['light_regime','plate', 'count'])

# Show result
summary

,light_regime,plate,count,n_mutants
0,10min-10min,32v1,1,363
1,10min-10min,32v1,2,12
2,10min-10min,32v1,6,1
3,10min-10min,32v2,1,373
4,10min-10min,32v2,2,2
5,10min-10min,32v2,6,1
6,10min-10min,32v3,1,373
7,10min-10min,32v3,2,2
8,10min-10min,32v3,6,1
9,1min-1min,32v1,1,373


In [77]:
from scipy import interpolate
from scipy.stats import rankdata

def normalize_quantiles(A, ties=True):
    A = np.asarray(A, dtype=np.float64)
    n_rows, n_cols = A.shape
    if n_cols == 1:
        return A.copy()

    i = np.linspace(0, 1, n_rows)
    S = np.full((n_rows, n_cols), np.nan)
    nobs = np.zeros(n_cols, dtype=int)
    sort_idx = []

    for j in range(n_cols):
        col = A[:, j]
        not_nan = ~np.isnan(col)
        x = col[not_nan]
        nobs[j] = len(x)
        sort_order = np.argsort(x)
        sorted_x = x[sort_order]

        if nobs[j] < n_rows:
            f = interpolate.interp1d(np.linspace(0, 1, nobs[j]), sorted_x,
                                     bounds_error=False, fill_value="extrapolate")
            S[:, j] = f(i)
        else:
            S[:, j] = sorted_x

        sort_idx.append(np.argsort(np.argsort(col[not_nan])))

    m = np.nanmean(S, axis=1)
    A_out = np.full_like(A, np.nan)

    for j in range(n_cols):
        col = A[:, j]
        not_nan = ~np.isnan(col)

        if ties:
            r = rankdata(col[not_nan], method='average')
            quant_pos = (r - 1) / (nobs[j] - 1)
            f = interpolate.interp1d(i, m, bounds_error=False, fill_value="extrapolate")
            A_out[not_nan, j] = f(quant_pos)
        else:
            ranks = sort_idx[j]
            A_out[not_nan, j] = m[ranks.astype(int)]

    return A_out

In [78]:
# Step 1: Filter data for the three plates
pd.set_option('display.max_rows', 80)
plates = ['32v1', '32v2', '32v3']
df_32v =phase2_df1[phase2_df1['plate'].isin(plates)]

# Step 2: Count rows per (plate, mutant_ID, mutated_genes, light_regime)
group_counts = (
    df_30v.groupby(['plate', 'light_regime', 'mutant_ID', 'mutated_genes'])
    .size()
    .reset_index(name='count')
)

# Step 3: For each plate and light_regime, count how many mutants had 1, 2, ... rows
summary = (
    group_counts.groupby(['light_regime','plate', 'count'])
    .size()
    .reset_index(name='n_mutants')
)

# Optional: Sort for easier reading
summary = summary.sort_values(by=['light_regime','plate', 'count'])

# Show result
summary

,light_regime,plate,count,n_mutants
0,10min-10min,32v1,2,373
1,10min-10min,32v1,4,2
2,10min-10min,32v1,12,1
3,10min-10min,32v2,1,373
4,10min-10min,32v2,2,2
5,10min-10min,32v2,6,1
6,10min-10min,32v3,1,373
7,10min-10min,32v3,2,2
8,10min-10min,32v3,6,1
9,1min-1min,32v1,1,373


In [9]:
df_32v[(df_32v['mutant_ID']=='WT')&(df_32v['plate']=='32v1')&(df_32v['light_regime']=='10min-10min')]

,plate,measurement,start_date,light_regime,dark_threshold,light_threshold,num_frames,i,j,fv_fm,...,measurement_time_173,measurement_time_174,measurement_time_175,measurement_time_176,measurement_time_177,well_id,mutant_ID,feature,mutated_genes,num_mutations
11922,32v1,M6,2024-08-09,10min-10min,15.521382,21.828480,180,2,2,0.655346,...,NaN,NaN,NaN,NaN,NaN,C03,WT,NaN,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0
11931,32v1,M6,2024-08-09,10min-10min,15.521382,21.828480,180,2,11,0.667968,...,NaN,NaN,NaN,NaN,NaN,C12,WT,NaN,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0
11941,32v1,M6,2024-08-09,10min-10min,15.521382,21.828480,180,2,21,0.636650,...,NaN,NaN,NaN,NaN,NaN,C22,WT,NaN,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0
12186,32v1,M6,2024-08-09,10min-10min,15.521382,21.828480,180,13,2,0.624098,...,NaN,NaN,NaN,NaN,NaN,N03,WT,NaN,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0
12195,32v1,M6,2024-08-09,10min-10min,15.521382,21.828480,180,13,11,0.634073,...,NaN,NaN,NaN,NaN,NaN,N12,WT,NaN,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0
12205,32v1,M6,2024-08-09,10min-10min,15.521382,21.828480,180,13,21,0.558001,...,NaN,NaN,NaN,NaN,NaN,N22,WT,NaN,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0
12305,32v1,M6,2024-11-20,10min-10min,15.444170,21.656998,172,2,2,0.677334,...,NaN,NaN,NaN,NaN,NaN,C03,WT,NaN,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0
12314,32v1,M6,2024-11-20,10min-10min,15.444170,21.656998,172,2,11,0.694387,...,NaN,NaN,NaN,NaN,NaN,C12,WT,NaN,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0
12324,32v1,M6,2024-11-20,10min-10min,15.444170,21.656998,172,2,21,0.694739,...,NaN,NaN,NaN,NaN,NaN,C22,WT,NaN,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0
12569,32v1,M6,2024-11-20,10min-10min,15.444170,21.656998,172,13,2,0.682515,...,NaN,NaN,NaN,NaN,NaN,N03,WT,NaN,"Cre09.g392000,Cre09.g406500,Cre02.g106400",3.0


## 32 plate 2h-ML 

In [10]:
def quantile_normalize_light_regime(df, light_regime, plates, y2_cols, tie_handling=True):
    """
    Quantile-normalize all y2_cols across selected plates within a given light regime.
    
    """
    # Filter data
    subset_df = df[(df['light_regime'] == light_regime) & (df['plate'].isin(plates))].copy()
    df_normalized = subset_df.copy()

    for timepoint in y2_cols:
        position_values = []
        valid_plate_indices = {}

        for plate in plates:
            plate_df = subset_df[subset_df['plate'] == plate].copy()

            wt_rows = plate_df[plate_df['mutant_ID'] == 'WT'].copy()
            non_wt_rows = plate_df[plate_df['mutant_ID'] != 'WT'].copy()

            wt_rows = wt_rows.sort_values(['mutant_ID', 'mutated_genes', 'well_id'])
            non_wt_rows = non_wt_rows.sort_values(['mutant_ID', 'mutated_genes'])

            sorted_df = pd.concat([wt_rows, non_wt_rows], axis=0)
            values = sorted_df[timepoint].values
            index = sorted_df.index.values

            position_values.append(values)
            valid_plate_indices[plate] = index

        # Validate shape
        lengths = [len(v) for v in position_values]
        if len(set(lengths)) != 1:
            raise ValueError(f"Length mismatch at {timepoint}: {lengths}")

        matrix = np.column_stack(position_values)
        normalized_matrix = normalize_quantiles(matrix, ties=tie_handling)

        # Write back
        for col_idx, plate in enumerate(plates):
            indices = valid_plate_indices[plate]
            df_normalized.loc[indices, timepoint] = normalized_matrix[:, col_idx]

    return df_normalized

In [11]:
# Define inputs
plates = ['32v1', '32v2', '32v3']
y2_cols = [f'y2_{i}' for i in range(1, 45)]

# Run normalization
phase2_32_20h_ML_normalized = quantile_normalize_light_regime(
    df=phase2_df1,
    light_regime='20h_ML',
    plates=plates,
    y2_cols=y2_cols
)

# View a few columns
phase2_32_20h_ML_normalized[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_35,y2_36,y2_37,y2_38,y2_39,y2_40,y2_41,y2_42,y2_43,y2_44
5362,32v3,LMJ.RY0402.043414,Cre07.g340400,A02,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5363,32v3,LMJ.RY0402.253077,Cre07.g329750,A03,0.251305,0.287736,0.329875,0.249636,0.213918,0.294279,...,0.233432,0.229617,0.207688,0.188170,0.236610,0.199026,0.223139,0.211330,0.259972,0.212252
5364,32v3,LMJ.RY0402.093716,Cre01.g036100,A04,0.401773,0.422689,0.447005,0.441124,0.443885,0.458794,...,0.387735,0.380214,0.409939,0.431461,0.392722,0.406955,0.412748,0.393926,0.413848,0.392536
5365,32v3,LMJ.RY0402.252604,Cre08.g362000,A05,0.354230,0.392665,0.418035,0.442344,0.430385,0.389412,...,0.348082,0.365530,0.355850,0.358670,0.368866,0.344215,0.365077,0.364001,0.368346,0.355651
5366,32v3,LMJ.RY0402.250044,Cre06.g269801,A06,0.412485,0.420053,0.428216,0.433277,0.448103,0.425193,...,0.392843,0.393370,0.400907,0.383400,0.399636,0.413633,0.426688,0.386312,0.380638,0.400741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32167,32v1,LMJ.RY0402.048478,Cre06.g270450,P20,0.341509,0.397976,0.376721,0.388842,0.354644,0.381056,...,0.396873,0.357741,0.396494,0.314344,0.349061,0.305550,0.345511,0.357289,0.333337,0.364779
32168,32v1,LMJ.RY0402.065081,Cre12.g493050,P21,0.440204,0.512606,0.503182,0.491149,0.522407,0.486903,...,0.480638,0.476015,0.471049,0.474150,0.477732,0.456994,0.483707,0.461696,0.469976,0.462918
32169,32v1,LMJ.RY0402.165706,Cre02.g098350,P22,0.293563,0.285296,0.293501,0.272002,0.289242,0.293367,...,0.272188,0.233410,0.285042,0.277435,0.229909,0.229079,0.248405,0.236902,0.249830,0.234159
32170,32v1,LMJ.RY0402.068059,Cre02.g098350,P23,0.304941,0.302155,0.356646,0.334831,0.325534,0.337765,...,0.331703,0.331803,0.284471,0.317492,0.326958,0.314438,0.288047,0.272809,0.297905,0.335465


In [12]:
plates = ['32v1', '32v2','32v3']
phase2_32_20h_ML= phase2_df1[(phase2_df1['light_regime'] == '20h_ML') & (phase2_df1['plate'].isin(plates))].copy()
phase2_32_20h_ML[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols[:10]]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,y2_7,y2_8,y2_9,y2_10
5362,32v3,LMJ.RY0402.043414,Cre07.g340400,A02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5363,32v3,LMJ.RY0402.253077,Cre07.g329750,A03,0.269970,0.293005,0.328458,0.280972,0.241560,0.304293,0.300955,0.309354,0.258212,0.250391
5364,32v3,LMJ.RY0402.093716,Cre01.g036100,A04,0.411952,0.435583,0.454215,0.454588,0.456940,0.469501,0.473639,0.486351,0.448351,0.456258
5365,32v3,LMJ.RY0402.252604,Cre08.g362000,A05,0.362409,0.403583,0.423892,0.455456,0.447064,0.396039,0.397383,0.401843,0.411313,0.418273
5366,32v3,LMJ.RY0402.250044,Cre06.g269801,A06,0.424591,0.432840,0.435389,0.445921,0.463614,0.434016,0.467612,0.453342,0.435353,0.418014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32167,32v1,LMJ.RY0402.048478,Cre06.g270450,P20,0.333859,0.392427,0.372982,0.380526,0.338695,0.374140,0.394284,0.359182,0.334045,0.431763
32168,32v1,LMJ.RY0402.065081,Cre12.g493050,P21,0.434598,0.507603,0.497651,0.483610,0.504738,0.478906,0.500026,0.506800,0.505400,0.520342
32169,32v1,LMJ.RY0402.165706,Cre02.g098350,P22,0.286175,0.285279,0.286123,0.276001,0.281851,0.279631,0.299724,0.290459,0.308943,0.284281
32170,32v1,LMJ.RY0402.068059,Cre02.g098350,P23,0.297711,0.302452,0.352578,0.329400,0.316222,0.331851,0.305923,0.356321,0.317455,0.336293


## plate 32 20_HL

In [13]:
# Define inputs
plates = ['32v1', '32v2', '32v3']
y2_cols = [f'y2_{i}' for i in range(1, 45)]

# Run normalization
phase2_32_20h_HL_normalized = quantile_normalize_light_regime(
    df=phase2_df1,
    light_regime='20h_HL',
    plates=plates,
    y2_cols=y2_cols
)

# View a few columns
phase2_32_20h_HL_normalized[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_35,y2_36,y2_37,y2_38,y2_39,y2_40,y2_41,y2_42,y2_43,y2_44
1149,32v3,LMJ.RY0402.043414,Cre07.g340400,A02,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1150,32v3,LMJ.RY0402.253077,Cre07.g329750,A03,0.157163,0.125408,0.067323,0.054848,0.179196,0.111494,...,0.032539,0.074807,0.039387,-0.000712,0.020502,0.037664,0.073583,0.075493,0.062374,0.016244
1151,32v3,LMJ.RY0402.093716,Cre01.g036100,A04,0.309436,0.268989,0.306509,0.330658,0.323757,0.305775,...,0.143357,0.140612,0.136002,0.155704,0.177462,0.158003,0.174998,0.157022,0.125808,0.167198
1152,32v3,LMJ.RY0402.252604,Cre08.g362000,A05,0.165518,0.256549,0.149090,0.208872,0.264669,0.205863,...,0.075251,0.069064,0.086866,0.076208,0.038325,0.163848,0.076413,0.125667,0.100703,0.058646
1153,32v3,LMJ.RY0402.250044,Cre06.g269801,A06,0.270385,0.282240,0.238637,0.201940,0.247855,0.217903,...,0.107175,0.139761,0.108467,0.103821,0.124380,0.102546,0.124134,0.135924,0.099540,0.137949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33699,32v2,LMJ.RY0402.243149,Cre16.g688350,P20,0.328539,0.335359,0.328431,0.298095,0.316470,0.206131,...,0.068568,0.086271,0.085947,0.096321,0.060798,0.071806,0.038365,0.059803,0.091977,0.036315
33700,32v2,LMJ.RY0402.057143,Cre02.g095063,P21,0.162921,0.261821,0.173605,0.253647,0.205131,0.116985,...,0.098186,0.113375,0.023456,0.130801,0.090109,0.123797,0.105075,0.138547,0.078022,0.125191
33701,32v2,LMJ.RY0402.140370,Cre07.g325754,P22,0.188614,0.199824,0.141827,0.218335,0.214396,0.283206,...,0.191610,0.137669,0.114131,0.135253,0.163995,0.155754,0.098760,0.139833,0.190767,0.169677
33702,32v2,LMJ.RY0402.248337,Cre01.g009700,P23,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
plates = ['32v1', '32v2','32v3']
phase2_32_20h_HL= phase2_df1[(phase2_df1['light_regime'] == '20h_HL') & (phase2_df1['plate'].isin(plates))].copy()
phase2_32_20h_HL[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols[:10]]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,y2_7,y2_8,y2_9,y2_10
1149,32v3,LMJ.RY0402.043414,Cre07.g340400,A02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1150,32v3,LMJ.RY0402.253077,Cre07.g329750,A03,0.160698,0.148241,0.077550,0.077568,0.190312,0.133281,0.124948,0.167897,0.084976,0.130591
1151,32v3,LMJ.RY0402.093716,Cre01.g036100,A04,0.310682,0.278510,0.307905,0.339302,0.329591,0.324895,0.320253,0.282779,0.328483,0.324877
1152,32v3,LMJ.RY0402.252604,Cre08.g362000,A05,0.166152,0.265066,0.155756,0.223204,0.272891,0.224947,0.190852,0.255810,0.137493,0.281461
1153,32v3,LMJ.RY0402.250044,Cre06.g269801,A06,0.269218,0.294342,0.237770,0.214196,0.257255,0.235875,0.193795,0.250974,0.232592,0.231792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33699,32v2,LMJ.RY0402.243149,Cre16.g688350,P20,0.339927,0.333500,0.342579,0.304020,0.315760,0.207622,0.288040,0.329365,0.319298,0.246565
33700,32v2,LMJ.RY0402.057143,Cre02.g095063,P21,0.175929,0.268843,0.184912,0.259568,0.215518,0.115492,0.206496,0.236985,0.163897,0.190299
33701,32v2,LMJ.RY0402.140370,Cre07.g325754,P22,0.201253,0.206191,0.152256,0.229325,0.223274,0.282271,0.293496,0.279664,0.220971,0.172490
33702,32v2,LMJ.RY0402.248337,Cre01.g009700,P23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## plate 32 2h - 2h

In [17]:
# Define inputs
plates = ['32v1', '32v2','32v3']
y2_cols = [f'y2_{i}' for i in range(1, 49)]

# Run normalization
phase2_32_2h_2h_normalized = quantile_normalize_light_regime(
    df=phase2_df1,
    light_regime='2h-2h',
    plates=plates,
    y2_cols=y2_cols
)

# View a few columns
phase2_32_2h_2h_normalized[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_39,y2_40,y2_41,y2_42,y2_43,y2_44,y2_45,y2_46,y2_47,y2_48
7277,32v3,LMJ.RY0402.043414,Cre07.g340400,A02,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7278,32v3,LMJ.RY0402.253077,Cre07.g329750,A03,0.101541,0.081491,0.102234,0.069527,0.075303,0.111363,...,0.475426,0.475703,0.112018,0.066101,0.109869,0.033348,0.402232,0.461979,0.474833,0.461239
7279,32v3,LMJ.RY0402.093716,Cre01.g036100,A04,0.288711,0.257551,0.299845,0.290230,0.298346,0.256998,...,0.670427,0.644750,0.225626,0.208490,0.264773,0.225044,0.645327,0.666601,0.672177,0.659407
7280,32v3,LMJ.RY0402.252604,Cre08.g362000,A05,0.227704,0.231678,0.230711,0.239948,0.262073,0.232944,...,0.630616,0.647131,0.143916,0.186306,0.162610,0.185707,0.625859,0.624385,0.643795,0.632669
7281,32v3,LMJ.RY0402.250044,Cre06.g269801,A06,0.205577,0.237913,0.205069,0.195263,0.226278,0.175162,...,0.668365,0.651531,0.171606,0.188908,0.179631,0.178103,0.653475,0.663108,0.660114,0.657375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27954,32v2,LMJ.RY0402.243149,Cre16.g688350,P20,0.363744,0.312076,0.328821,0.318230,0.332819,0.274658,...,0.675642,0.676184,0.210224,0.161147,0.172726,0.189366,0.647283,0.673341,0.674404,0.693090
27955,32v2,LMJ.RY0402.057143,Cre02.g095063,P21,0.218043,0.215579,0.236290,0.195967,0.204674,0.256752,...,0.618689,0.607091,0.199267,0.208163,0.235504,0.178138,0.609987,0.621776,0.613552,0.635094
27956,32v2,LMJ.RY0402.140370,Cre07.g325754,P22,0.265392,0.186227,0.282724,0.228665,0.194688,0.185851,...,0.585883,0.610116,0.195219,0.243787,0.174641,0.247331,0.588177,0.609194,0.593575,0.589721
27957,32v2,LMJ.RY0402.248337,Cre01.g009700,P23,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
plates = ['32v1', '32v2','32v3']
phase2_32_2h_2h= phase2_df1[(phase2_df1['light_regime'] == '2h-2h') & (phase2_df1['plate'].isin(plates))].copy()
phase2_32_2h_2h[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols[:10]]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,y2_7,y2_8,y2_9,y2_10
7277,32v3,LMJ.RY0402.043414,Cre07.g340400,A02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7278,32v3,LMJ.RY0402.253077,Cre07.g329750,A03,0.095924,0.085938,0.099128,0.086422,0.085058,0.111463,0.125901,0.050756,0.453688,0.491160
7279,32v3,LMJ.RY0402.093716,Cre01.g036100,A04,0.293768,0.261294,0.297687,0.310003,0.302963,0.256800,0.271996,0.281250,0.612560,0.646939
7280,32v3,LMJ.RY0402.252604,Cre08.g362000,A05,0.226179,0.232968,0.230850,0.261379,0.269307,0.234547,0.213286,0.219184,0.590917,0.624645
7281,32v3,LMJ.RY0402.250044,Cre06.g269801,A06,0.205696,0.240122,0.204294,0.214654,0.227266,0.175921,0.225626,0.193518,0.624346,0.637957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27954,32v2,LMJ.RY0402.243149,Cre16.g688350,P20,0.355859,0.289867,0.301927,0.272887,0.288687,0.250397,0.272966,0.233381,0.645780,0.690969
27955,32v2,LMJ.RY0402.057143,Cre02.g095063,P21,0.210717,0.206330,0.219784,0.164735,0.176119,0.235396,0.152244,0.129355,0.533706,0.580052
27956,32v2,LMJ.RY0402.140370,Cre07.g325754,P22,0.252849,0.177099,0.255769,0.192203,0.166746,0.167650,0.198809,0.194440,0.555214,0.599836
27957,32v2,LMJ.RY0402.248337,Cre01.g009700,P23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 32 plate 10min-10min

In [85]:
subset[subset['light_regime']=='10min-10min']['plate'].unique()

array(['30v2', '30v3', '31v3', '33v1', '32v1', '30v1', '31v1', '32v3',
       '32v2', '33v2', '31v2'], dtype=object)

In [88]:
# Define inputs
plates = ['32v1','32v2','32v3']
y2_cols = [f'y2_{i}' for i in range(1, 85)]

# Run normalization
subset=phase2_df1[~((phase2_df1['light_regime']=='10min-10min')&(phase2_df1['plate']=='32v1')&(phase2_df1['start_date']=='2024-08-09'))]
phase2_32_10min_10min_normalized = quantile_normalize_light_regime(
    df=subset,
    light_regime='10min-10min',
    plates=plates,
    y2_cols=y2_cols
)

# View a few columns
phase2_32_10min_10min_normalized[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_75,y2_76,y2_77,y2_78,y2_79,y2_80,y2_81,y2_82,y2_83,y2_84
12256,32v1,LMJ.RY0402.039093,Cre13.g588959,A02,0.265916,0.601813,0.274191,0.604352,0.277615,0.612696,...,0.229605,0.573681,0.593915,0.209701,0.183716,0.583514,0.619035,0.160004,0.263897,0.620406
12257,32v1,LMJ.RY0402.039746,Cre02.g080900,A03,0.116712,0.515278,0.177911,0.554415,0.126631,0.524153,...,0.115697,0.540315,0.540903,0.155783,0.190988,0.532146,0.549910,0.173164,0.157735,0.548854
12258,32v1,LMJ.RY0402.039953,Cre02.g095137,A04,0.265133,0.584804,0.275487,0.586717,0.274432,0.605488,...,0.205588,0.566890,0.541924,0.244467,0.228501,0.576098,0.564258,0.172222,0.188675,0.572014
12259,32v1,LMJ.RY0402.040368,Cre06.g278251,A05,0.186056,0.531968,0.133181,0.533009,0.137873,0.551641,...,0.145299,0.543873,0.569940,0.130754,0.136960,0.554145,0.586910,0.121758,0.172595,0.553563
12260,32v1,LMJ.RY0402.040792,Cre02.g098750,A06,0.095900,0.435042,0.052444,0.428606,0.099977,0.473744,...,0.057214,0.449434,0.468727,0.044669,0.039402,0.437390,0.451481,0.059624,0.101137,0.465905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31018,32v2,LMJ.RY0402.243149,Cre16.g688350,P20,0.244496,0.668915,0.230572,0.618034,0.316613,0.635568,...,0.235594,0.640593,0.639849,0.202261,0.139682,0.636386,0.649159,0.198646,0.227107,0.640080
31019,32v2,LMJ.RY0402.057143,Cre02.g095063,P21,0.252666,0.600101,0.212984,0.616015,0.154556,0.557888,...,0.211769,0.581183,0.572874,0.141647,0.130195,0.552295,0.590608,0.173408,0.215270,0.593482
31020,32v2,LMJ.RY0402.140370,Cre07.g325754,P22,0.137993,0.583702,0.215498,0.587119,0.213978,0.590450,...,0.245685,0.577386,0.593675,0.189682,0.181895,0.589526,0.601993,0.127730,0.116970,0.622734
31021,32v2,LMJ.RY0402.248337,Cre01.g009700,P23,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
plates = ['32v1', '32v2','32v3']
phase2_32_10min_10min= subset[(subset['light_regime'] == '10min-10min') & (subset['plate'].isin(plates))].copy()
phase2_32_10min_10min[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_75,y2_76,y2_77,y2_78,y2_79,y2_80,y2_81,y2_82,y2_83,y2_84
12256,32v1,LMJ.RY0402.039093,Cre13.g588959,A02,0.219566,0.607292,0.219560,0.600911,0.220074,0.607097,...,0.157615,0.582513,0.597039,0.140630,0.130838,0.588998,0.607419,0.116089,0.184735,0.612898
12257,32v1,LMJ.RY0402.039746,Cre02.g080900,A03,0.127561,0.542836,0.160912,0.566810,0.122559,0.546330,...,0.090818,0.555090,0.560084,0.110369,0.134525,0.548479,0.564581,0.125281,0.112826,0.563691
12258,32v1,LMJ.RY0402.039953,Cre02.g095137,A04,0.219223,0.594898,0.219983,0.591333,0.217484,0.603630,...,0.146027,0.578724,0.560672,0.162224,0.157480,0.584099,0.577790,0.124975,0.133577,0.581275
12259,32v1,LMJ.RY0402.040368,Cre06.g278251,A05,0.175564,0.554108,0.129692,0.548867,0.130718,0.566259,...,0.108674,0.560167,0.582180,0.096042,0.100453,0.564548,0.593021,0.088742,0.122156,0.567621
12260,32v1,LMJ.RY0402.040792,Cre02.g098750,A06,0.111104,0.482539,0.085763,0.466140,0.096964,0.508670,...,0.037464,0.490062,0.498594,0.021862,0.029740,0.502705,0.490336,0.040178,0.072959,0.497550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31018,32v2,LMJ.RY0402.243149,Cre16.g688350,P20,0.250586,0.681778,0.236383,0.622720,0.347597,0.643985,...,0.269178,0.649529,0.650186,0.237309,0.162273,0.648433,0.662491,0.227206,0.264755,0.649781
31019,32v2,LMJ.RY0402.057143,Cre02.g095063,P21,0.262980,0.600246,0.219033,0.621269,0.159767,0.556345,...,0.242956,0.576999,0.572204,0.165417,0.151368,0.550272,0.591449,0.199866,0.251105,0.597794
31020,32v2,LMJ.RY0402.140370,Cre07.g325754,P22,0.131782,0.582098,0.221942,0.586982,0.228527,0.591582,...,0.277000,0.572922,0.597728,0.219816,0.208485,0.590209,0.604480,0.147548,0.137891,0.630002
31021,32v2,LMJ.RY0402.248337,Cre01.g009700,P23,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 32 plate 1min 1min

In [23]:
# Define inputs
plates = ['32v1', '32v2','32v3']
y2_cols = [f'y2_{i}' for i in range(1, 89)]

# Run normalization
phase2_32_1min_1min_normalized = quantile_normalize_light_regime(
    df=phase2_df1,
    light_regime='1min-1min',
    plates=plates,
    y2_cols=y2_cols
)

# View a few columns
phase2_32_1min_1min_normalized[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_79,y2_80,y2_81,y2_82,y2_83,y2_84,y2_85,y2_86,y2_87,y2_88
8043,32v1,LMJ.RY0402.039093,Cre13.g588959,A02,0.217715,0.546479,0.223315,0.532274,0.178789,0.534455,...,0.161242,0.492952,0.098079,0.501484,0.140850,0.446055,0.168070,0.492321,0.149834,0.476492
8044,32v1,LMJ.RY0402.039746,Cre02.g080900,A03,0.157381,0.488439,0.130626,0.455894,0.131504,0.518910,...,0.153223,0.487003,0.070136,0.481176,0.106667,0.443138,0.105074,0.468930,0.112177,0.461361
8045,32v1,LMJ.RY0402.039953,Cre02.g095137,A04,0.249791,0.550402,0.251349,0.526057,0.191816,0.536478,...,0.182195,0.522461,0.210129,0.486116,0.157914,0.513864,0.156715,0.506827,0.193901,0.495003
8046,32v1,LMJ.RY0402.040368,Cre06.g278251,A05,0.071607,0.489811,0.144940,0.474464,0.118319,0.472089,...,0.092710,0.430106,0.092808,0.453123,0.121049,0.418393,0.100148,0.460547,0.084261,0.450008
8047,32v1,LMJ.RY0402.040792,Cre02.g098750,A06,0.103630,0.440915,0.051039,0.425809,0.105915,0.426689,...,0.063971,0.368046,0.039164,0.374396,0.068169,0.333489,0.021736,0.343677,0.045850,0.354084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21060,32v3,LMJ.RY0402.199716,Cre14.g621800,P20,0.161298,0.510752,0.116752,0.512202,0.164455,0.466862,...,0.099976,0.460585,0.155498,0.403452,0.116531,0.418393,0.086162,0.398118,0.138214,0.475209
21061,32v3,LMJ.RY0402.056457,Cre09.g414650,P21,0.158134,0.503405,0.140107,0.447630,0.148256,0.476558,...,0.139933,0.468571,0.132219,0.449841,0.143976,0.456425,0.111214,0.476263,0.106062,0.429241
21062,32v3,LMJ.RY0402.208970,Cre07.g314800,P22,0.193325,0.524521,0.204398,0.487404,0.207248,0.487691,...,0.119054,0.462618,0.066397,0.480287,0.092881,0.432770,0.133064,0.407596,0.135519,0.490043
21063,32v3,LMJ.RY0402.193328,Cre17.g720261,P23,0.254842,0.608189,0.271119,0.571680,0.257660,0.566854,...,0.202533,0.519080,0.201757,0.478424,0.205512,0.488466,0.178525,0.557508,0.221235,0.548136


In [24]:
plates = ['32v1', '32v2','32v3']
phase2_32_1min_1min= phase2_df1[(phase2_df1['light_regime'] == '1min-1min') & (phase2_df1['plate'].isin(plates))].copy()
phase2_32_1min_1min[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_79,y2_80,y2_81,y2_82,y2_83,y2_84,y2_85,y2_86,y2_87,y2_88
8043,32v1,LMJ.RY0402.039093,Cre13.g588959,A02,0.202436,0.533086,0.199285,0.519535,0.156386,0.523333,...,0.136775,0.465928,0.075863,0.483230,0.121485,0.427227,0.146542,0.475057,0.124707,0.460690
8044,32v1,LMJ.RY0402.039746,Cre02.g080900,A03,0.141499,0.476958,0.103579,0.437662,0.107005,0.505921,...,0.128896,0.459810,0.044384,0.461497,0.083487,0.423334,0.081704,0.448480,0.082954,0.446646
8045,32v1,LMJ.RY0402.039953,Cre02.g095137,A04,0.233858,0.537572,0.229277,0.512299,0.169666,0.523894,...,0.161174,0.497308,0.194471,0.466288,0.134733,0.497406,0.136601,0.488689,0.173369,0.479915
8046,32v1,LMJ.RY0402.040368,Cre06.g278251,A05,0.067067,0.478227,0.117594,0.454138,0.090157,0.462422,...,0.063509,0.394472,0.069552,0.436979,0.098206,0.400083,0.077506,0.437094,0.056985,0.435151
8047,32v1,LMJ.RY0402.040792,Cre02.g098750,A06,0.090473,0.421974,0.021365,0.408502,0.078222,0.400880,...,0.034941,0.342549,0.014078,0.361587,0.049600,0.297388,0.018041,0.303321,0.028565,0.324647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21060,32v3,LMJ.RY0402.199716,Cre14.g621800,P20,0.165112,0.512962,0.124486,0.519465,0.179494,0.473093,...,0.115742,0.474999,0.165513,0.423185,0.128832,0.433531,0.099595,0.418128,0.148103,0.482396
21061,32v3,LMJ.RY0402.056457,Cre09.g414650,P21,0.163805,0.508217,0.149226,0.452258,0.164287,0.481581,...,0.152842,0.480450,0.141307,0.463050,0.153077,0.463581,0.122845,0.481899,0.123473,0.444142
21062,32v3,LMJ.RY0402.208970,Cre07.g314800,P22,0.194924,0.524702,0.210748,0.494306,0.220651,0.494605,...,0.134775,0.476612,0.088303,0.491494,0.108984,0.448600,0.140439,0.424193,0.143721,0.496079
21063,32v3,LMJ.RY0402.193328,Cre17.g720261,P23,0.254538,0.603972,0.271884,0.575243,0.266362,0.569314,...,0.214812,0.525009,0.211095,0.490296,0.216499,0.493271,0.187439,0.555161,0.225941,0.548658


## plate 32 30s-30s

In [26]:
# Define inputs
plates = ['32v1','32v2', '32v3']
y2_cols = [f'y2_{i}' for i in range(1, 89)]

# Run normalization
phase2_32_30s_30s_normalized = quantile_normalize_light_regime(
    df=phase2_df1,
    light_regime='30s-30s',
    plates=plates,
    y2_cols=y2_cols
)

# View a few columns
phase2_32_30s_30s_normalized[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_79,y2_80,y2_81,y2_82,y2_83,y2_84,y2_85,y2_86,y2_87,y2_88
8809,32v2,LMJ.RY0402.251080,Cre14.g632750,A02,0.273633,0.564989,0.236029,0.577784,0.285474,0.450319,...,0.249361,0.491833,0.312722,0.503074,0.164591,0.467645,0.291435,0.503363,0.171128,0.495143
8810,32v2,LMJ.RY0402.234983,Cre02.g095126,A03,0.149320,0.505218,0.233044,0.542205,0.170874,0.460426,...,0.191458,0.481626,0.212113,0.446256,0.201344,0.479781,0.220659,0.500220,0.168640,0.477587
8811,32v2,LMJ.RY0402.193328,Cre17.g720261,A04,0.274317,0.574635,0.265746,0.575680,0.286777,0.534001,...,0.268493,0.547185,0.223878,0.524249,0.272791,0.567454,0.255594,0.567220,0.240872,0.574406
8812,32v2,LMJ.RY0402.146540,Cre10.g434500,A05,0.215110,0.544309,0.224528,0.513917,0.201413,0.490816,...,0.196621,0.491343,0.173429,0.497698,0.200305,0.470999,0.175260,0.510824,0.183546,0.506477
8813,32v2,LMJ.RY0402.234933,Cre01.g022900,A06,0.264752,0.575388,0.276887,0.556200,0.252770,0.494927,...,0.233226,0.546011,0.198909,0.476671,0.178107,0.518467,0.173710,0.542467,0.178771,0.532486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21826,32v3,LMJ.RY0402.199716,Cre14.g621800,P20,0.199136,0.446629,0.060074,0.386942,0.156242,0.490891,...,0.053764,0.460158,0.182189,0.472902,0.160170,0.520248,0.153445,0.445634,-0.005877,0.449794
21827,32v3,LMJ.RY0402.056457,Cre09.g414650,P21,0.219411,0.532032,0.206900,0.470667,0.182144,0.449256,...,0.198228,0.470254,0.190596,0.509766,0.170579,0.482018,0.137138,0.463973,0.145261,0.538782
21828,32v3,LMJ.RY0402.208970,Cre07.g314800,P22,0.149390,0.532564,0.153032,0.454233,0.276810,0.502601,...,0.023389,0.455894,0.192147,0.478606,0.139946,0.364187,0.158022,0.458849,0.031955,0.532048
21829,32v3,LMJ.RY0402.193328,Cre17.g720261,P23,0.320683,0.637214,0.340951,0.572944,0.256814,0.585458,...,0.343305,0.587574,0.302050,0.543803,0.304132,0.600849,0.261274,0.530571,0.207139,0.531053


In [27]:
plates = ['32v1', '32v2','32v3']
phase2_32_1min_1min= phase2_df1[(phase2_df1['light_regime'] == '30s-30s') & (phase2_df1['plate'].isin(plates))].copy()
phase2_32_1min_1min[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_79,y2_80,y2_81,y2_82,y2_83,y2_84,y2_85,y2_86,y2_87,y2_88
8809,32v2,LMJ.RY0402.251080,Cre14.g632750,A02,0.273480,0.565683,0.228947,0.579061,0.284918,0.449294,...,0.265999,0.496056,0.327332,0.512721,0.182868,0.479527,0.306044,0.503246,0.181313,0.498717
8810,32v2,LMJ.RY0402.234983,Cre02.g095126,A03,0.152198,0.501150,0.226179,0.547008,0.167589,0.463126,...,0.204927,0.487635,0.223787,0.456924,0.218523,0.489452,0.237479,0.500883,0.180113,0.478169
8811,32v2,LMJ.RY0402.193328,Cre17.g720261,A04,0.273523,0.576672,0.256611,0.577723,0.287147,0.532782,...,0.283926,0.546237,0.234655,0.532248,0.285864,0.573339,0.272713,0.567217,0.257794,0.575120
8812,32v2,LMJ.RY0402.146540,Cre10.g434500,A05,0.218474,0.544923,0.217898,0.514007,0.201498,0.492394,...,0.208860,0.496029,0.187190,0.504623,0.217004,0.481902,0.187580,0.508756,0.195549,0.508625
8813,32v2,LMJ.RY0402.234933,Cre01.g022900,A06,0.264629,0.577540,0.268063,0.560870,0.251623,0.495244,...,0.246986,0.545432,0.212444,0.485747,0.198837,0.524214,0.185691,0.541945,0.191039,0.534512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21826,32v3,LMJ.RY0402.199716,Cre14.g621800,P20,0.203658,0.445044,0.079422,0.368528,0.149821,0.488205,...,0.036938,0.462386,0.187042,0.471114,0.157088,0.525252,0.155374,0.439857,0.010271,0.454327
21827,32v3,LMJ.RY0402.056457,Cre09.g414650,P21,0.221705,0.531892,0.208900,0.461750,0.181364,0.447739,...,0.201320,0.471850,0.195329,0.511174,0.165481,0.483475,0.134826,0.458089,0.145926,0.541972
21828,32v3,LMJ.RY0402.208970,Cre07.g314800,P22,0.152184,0.532828,0.146319,0.446044,0.283091,0.501556,...,-0.026314,0.457997,0.197164,0.477592,0.131711,0.354420,0.161544,0.454890,0.032185,0.536537
21829,32v3,LMJ.RY0402.193328,Cre17.g720261,P23,0.332938,0.634067,0.346993,0.568567,0.259879,0.583929,...,0.350546,0.591430,0.308092,0.544480,0.314999,0.602449,0.265265,0.530428,0.213395,0.535876


## plate 32 5min 5min

In [29]:
# Define inputs
plates = ['32v1', '32v2','32v3']
y2_cols = [f'y2_{i}' for i in range(1, 89)]

# Run normalization
phase2_32_5min_5min_normalized = quantile_normalize_light_regime(
    df=phase2_df1,
    light_regime='5min-5min',
    plates=plates,
    y2_cols=y2_cols
)

# View a few columns
phase2_32_5min_5min_normalized[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_79,y2_80,y2_81,y2_82,y2_83,y2_84,y2_85,y2_86,y2_87,y2_88
9958,32v2,LMJ.RY0402.251080,Cre14.g632750,A02,0.239599,0.587860,0.264051,0.565840,0.188583,0.590974,...,0.220929,0.550286,0.187103,0.558130,0.174667,0.586147,0.201422,0.559636,0.157956,0.569267
9959,32v2,LMJ.RY0402.234983,Cre02.g095126,A03,0.173122,0.544987,0.163484,0.549076,0.162700,0.513805,...,0.155021,0.524270,0.114733,0.517648,0.102088,0.480106,0.074287,0.515917,0.087081,0.511329
9960,32v2,LMJ.RY0402.193328,Cre17.g720261,A04,0.214629,0.621614,0.213223,0.596248,0.205737,0.586672,...,0.215844,0.590387,0.202060,0.581648,0.181856,0.561229,0.201731,0.581812,0.183167,0.599567
9961,32v2,LMJ.RY0402.146540,Cre10.g434500,A05,0.219415,0.581386,0.224308,0.591999,0.205147,0.573351,...,0.176346,0.565658,0.199033,0.574564,0.157622,0.538074,0.169995,0.570958,0.162996,0.564708
9962,32v2,LMJ.RY0402.234933,Cre01.g022900,A06,0.277124,0.625116,0.278414,0.607435,0.232460,0.600062,...,0.210482,0.603210,0.209178,0.607538,0.238135,0.579368,0.215941,0.605433,0.214223,0.603119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32550,32v1,LMJ.RY0402.048478,Cre06.g270450,P20,0.144319,0.519119,0.152942,0.554211,0.094101,0.496654,...,0.100571,0.451839,0.129841,0.462833,0.077053,0.475221,0.039163,0.474693,0.051791,0.466514
32551,32v1,LMJ.RY0402.065081,Cre12.g493050,P21,0.265599,0.632715,0.235525,0.589658,0.241702,0.595383,...,0.237450,0.578850,0.215941,0.591597,0.222422,0.600845,0.141693,0.579219,0.212573,0.591602
32552,32v1,LMJ.RY0402.165706,Cre02.g098350,P22,0.128907,0.502444,0.135047,0.521859,0.104093,0.463108,...,0.072265,0.443988,0.119946,0.463775,0.047281,0.433277,0.064553,0.427574,0.090180,0.474253
32553,32v1,LMJ.RY0402.068059,Cre02.g098350,P23,0.209227,0.517026,0.216468,0.505307,0.106383,0.515999,...,0.119877,0.490686,0.114150,0.512864,0.110611,0.512108,0.156786,0.494321,0.103904,0.488988


In [28]:
plates = ['32v1', '32v2','32v3']
phase2_32_5min_5min= phase2_df1[(phase2_df1['light_regime'] == '5min-5min') & (phase2_df1['plate'].isin(plates))].copy()
phase2_32_5min_5min[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_79,y2_80,y2_81,y2_82,y2_83,y2_84,y2_85,y2_86,y2_87,y2_88
9958,32v2,LMJ.RY0402.251080,Cre14.g632750,A02,0.243438,0.587489,0.268337,0.567534,0.194052,0.597041,...,0.209425,0.544149,0.183101,0.551848,0.165467,0.577196,0.189854,0.554631,0.151418,0.564773
9959,32v2,LMJ.RY0402.234983,Cre02.g095126,A03,0.176587,0.544460,0.171728,0.552508,0.168149,0.519211,...,0.141267,0.519050,0.112977,0.512170,0.096169,0.470289,0.066051,0.507790,0.077919,0.506024
9960,32v2,LMJ.RY0402.193328,Cre17.g720261,A04,0.221002,0.620927,0.218683,0.598391,0.207871,0.592020,...,0.206059,0.583447,0.195444,0.575227,0.170088,0.553677,0.190155,0.575645,0.173988,0.591943
9961,32v2,LMJ.RY0402.146540,Cre10.g434500,A05,0.225046,0.582569,0.227489,0.594591,0.207428,0.578064,...,0.162895,0.559234,0.194060,0.569744,0.150394,0.531587,0.160240,0.564784,0.156035,0.561653
9962,32v2,LMJ.RY0402.234933,Cre01.g022900,A06,0.276497,0.624093,0.284015,0.611130,0.235136,0.604397,...,0.199096,0.596320,0.203415,0.602908,0.228425,0.569957,0.203098,0.599226,0.207986,0.595126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32550,32v1,LMJ.RY0402.048478,Cre06.g270450,P20,0.145064,0.523191,0.146312,0.559086,0.093599,0.499177,...,0.104310,0.456555,0.123992,0.472488,0.081504,0.478618,0.057019,0.480766,0.068798,0.467762
32551,32v1,LMJ.RY0402.065081,Cre12.g493050,P21,0.256646,0.629756,0.227001,0.592096,0.228531,0.590829,...,0.232313,0.574318,0.207965,0.589722,0.218323,0.596156,0.144108,0.579612,0.213592,0.591757
32552,32v1,LMJ.RY0402.165706,Cre02.g098350,P22,0.128926,0.507992,0.129375,0.526377,0.102531,0.471592,...,0.076788,0.445612,0.116376,0.472545,0.051865,0.453170,0.071810,0.437138,0.095077,0.475723
32553,32v1,LMJ.RY0402.068059,Cre02.g098350,P23,0.199220,0.520778,0.205126,0.512809,0.104711,0.515894,...,0.122010,0.489515,0.111999,0.516971,0.110752,0.512499,0.155199,0.498247,0.106405,0.491521


## plate 32 1min 5min¶

In [31]:
# Define inputs
plates = ['32v1','32v2', '32v3']
y2_cols = [f'y2_{i}' for i in range(1, 89)]

# Run normalization
phase2_32_1min_5min_normalized = quantile_normalize_light_regime(
    df=phase2_df1,
    light_regime='1min-5min',
    plates=plates,
    y2_cols=y2_cols
)

# View a few columns
phase2_32_1min_5min_normalized[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_79,y2_80,y2_81,y2_82,y2_83,y2_84,y2_85,y2_86,y2_87,y2_88
3830,32v2,LMJ.RY0402.251080,Cre14.g632750,A02,0.279335,0.622040,0.195380,0.627414,0.202275,0.643122,...,0.183719,0.581727,0.157308,0.603135,0.142082,0.602161,0.136158,0.617753,0.198616,0.619335
3831,32v2,LMJ.RY0402.234983,Cre02.g095126,A03,0.178157,0.574693,0.164611,0.555573,0.142025,0.555579,...,0.132260,0.535584,0.090457,0.550891,0.086383,0.561585,0.063018,0.553040,0.072648,0.563336
3832,32v2,LMJ.RY0402.193328,Cre17.g720261,A04,0.280459,0.637964,0.250450,0.616868,0.227016,0.617720,...,0.218990,0.606230,0.204394,0.619784,0.195200,0.621217,0.192688,0.632601,0.213376,0.613153
3833,32v2,LMJ.RY0402.146540,Cre10.g434500,A05,0.168198,0.595410,0.163207,0.559696,0.157462,0.575190,...,0.142382,0.594760,0.137241,0.595980,0.145997,0.593463,0.075769,0.589134,0.127901,0.583697
3834,32v2,LMJ.RY0402.234933,Cre01.g022900,A06,0.223608,0.642162,0.221225,0.616337,0.222799,0.604447,...,0.182828,0.618463,0.125501,0.614030,0.182530,0.610212,0.143360,0.607284,0.162799,0.598651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26422,32v1,LMJ.RY0402.048478,Cre06.g270450,P20,0.255471,0.611563,0.237679,0.579130,0.189194,0.580544,...,0.182494,0.593339,0.180429,0.592011,0.186182,0.569501,0.159444,0.587878,0.137272,0.574900
26423,32v1,LMJ.RY0402.065081,Cre12.g493050,P21,0.352919,0.677978,0.315261,0.652268,0.289774,0.659105,...,0.229779,0.639307,0.207594,0.651231,0.248882,0.640977,0.207311,0.640166,0.232157,0.634673
26424,32v1,LMJ.RY0402.165706,Cre02.g098350,P22,0.168391,0.548182,0.190238,0.523483,0.161636,0.532356,...,0.126208,0.536565,0.132501,0.560899,0.119006,0.511393,0.103337,0.501368,0.167718,0.519593
26425,32v1,LMJ.RY0402.068059,Cre02.g098350,P23,0.211429,0.610780,0.190750,0.610761,0.213356,0.582164,...,0.146386,0.576819,0.125847,0.597357,0.153772,0.579783,0.126478,0.557179,0.182558,0.578110


In [32]:
plates = ['32v1','32v2','32v3']
phase2_32_1min_5min= phase2_df1[(phase2_df1['light_regime'] == '1min-5min') & (phase2_df1['plate'].isin(plates))].copy()
phase2_32_1min_5min[['plate', 'mutant_ID', 'mutated_genes', 'well_id'] + y2_cols]

,plate,mutant_ID,mutated_genes,well_id,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,...,y2_79,y2_80,y2_81,y2_82,y2_83,y2_84,y2_85,y2_86,y2_87,y2_88
3830,32v2,LMJ.RY0402.251080,Cre14.g632750,A02,0.281609,0.628723,0.200140,0.632898,0.210985,0.644089,...,0.184690,0.584851,0.167680,0.604294,0.147836,0.604254,0.143470,0.618303,0.201853,0.619457
3831,32v2,LMJ.RY0402.234983,Cre02.g095126,A03,0.189921,0.579102,0.168849,0.566333,0.151694,0.559568,...,0.135157,0.538429,0.100786,0.552237,0.096562,0.564227,0.073918,0.552378,0.072826,0.564349
3832,32v2,LMJ.RY0402.193328,Cre17.g720261,A04,0.283714,0.641334,0.249982,0.623500,0.234366,0.621006,...,0.219888,0.610079,0.213683,0.617983,0.203678,0.623076,0.200694,0.631061,0.215367,0.613994
3833,32v2,LMJ.RY0402.146540,Cre10.g434500,A05,0.182236,0.600747,0.168623,0.568364,0.165014,0.579879,...,0.144345,0.597197,0.148262,0.598678,0.153561,0.599411,0.085403,0.590677,0.131425,0.584382
3834,32v2,LMJ.RY0402.234933,Cre01.g022900,A06,0.232478,0.644564,0.224993,0.623244,0.230084,0.608873,...,0.183917,0.621344,0.136163,0.613512,0.185526,0.612717,0.153538,0.607127,0.170482,0.598708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26422,32v1,LMJ.RY0402.048478,Cre06.g270450,P20,0.254769,0.609989,0.237205,0.576617,0.179948,0.575185,...,0.183739,0.591595,0.178927,0.588946,0.186857,0.568470,0.155325,0.592233,0.142505,0.578914
26423,32v1,LMJ.RY0402.065081,Cre12.g493050,P21,0.361300,0.683018,0.322445,0.653674,0.289277,0.656564,...,0.230841,0.638294,0.205548,0.651578,0.255440,0.642147,0.210781,0.647060,0.234306,0.639355
26424,32v1,LMJ.RY0402.165706,Cre02.g098350,P22,0.162231,0.549006,0.189677,0.525067,0.154757,0.527396,...,0.125260,0.535037,0.131003,0.556523,0.121877,0.512124,0.102542,0.512790,0.170287,0.525032
26425,32v1,LMJ.RY0402.068059,Cre02.g098350,P23,0.207048,0.608483,0.190196,0.607024,0.208108,0.576642,...,0.146071,0.574248,0.123232,0.594396,0.153701,0.577652,0.127623,0.561100,0.185136,0.581007


In [90]:
phase2_32_quantile1= pd.concat([
    phase2_32_20h_ML_normalized,
    phase2_32_20h_HL_normalized,
    phase2_32_2h_2h_normalized,
    phase2_32_10min_10min_normalized,
    phase2_32_1min_1min_normalized,
    phase2_32_30s_30s_normalized,
    phase2_32_5min_5min_normalized,
    phase2_32_1min_5min_normalized
], ignore_index=True)

In [92]:
phase2_32_quantile1.to_csv('phase2_32_qn(5.18).csv',index=False)

In [91]:
phase2_32_quantile1.shape

(9192, 726)

In [33]:
## drop a part of 10min-10min 32v1
plates = ['32v1', '32v2','32v3']
data=phase2_df1[phase2_df1['plate'].isin(plates)]
data.shape

(9575, 726)